# Skill extraction

In [18]:
import sys
import os
import pandas as pd

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.logger import setup_logging
from src.analysis.extracting_skills_list import SkillExtractor
from config.analysis import STANDARD_SKILL_MAP

setup_logging()

In [29]:
def test_skill_extractor(test_cases = [
        (
            "Precisamos de um profissional com experiência em Python e SQL.",
            ['Python', 'SQL']
        ),
        (
            "Conhecimento em Machine Learning (ML) e PowerBI é essencial.",
            ['Machine Learning', 'Power BI']
        ),
        (
            "Domínio de AWS, Azure e GCP para cloud computing.",
            ['Cloud', 'AWS', 'Azure', 'GCP']
        ),
        (
            "Experiência com PySpark e Spark para processamento de dados.",
            ['Spark']
        ),
        (
            "Habilidade em Excel avançado e análise de dados.",
            ['Excel']
        ),
        (
            "Conhecimento em Airflow para orquestração de pipelines.",
            ['Airflow']
        ),
        (
            "Não há requisitos técnicos específicos para esta vaga.",
            []
        ),
        (
            "Necessário conhecimento em Python e pandas para análise de dados.",
            ['Python', 'Pandas']
        ),
        (
            "A palavra 'excelente' não deve ser confundida com 3xc3l.",
            []
        ),
        (
            "MLOps e Machine Learning são importantes para a vaga.",
            ['Machine Learning']  # Assuming MLOps isn't in our test skills
        )
    ]):
    """Test function for SkillExtractor with various edge cases"""

    extractor = SkillExtractor(STANDARD_SKILL_MAP)
    
    passed = 0
    failed = 0
    failed_cases = []
    
    for text, expected in test_cases:
        try:
            result = extractor.extract_skills(text)
            result_set = set(result)
            expected_set = set(expected)
            
            # Check for missing skills
            missing = expected_set - result_set
            # Check for extra skills (only fail if we got completely unexpected skills)
            extra = result_set - expected_set
            print(extra)
            if not missing and not extra:
                passed += 1
                print(f"PASS: '{text}' -> {result}")
            else:
                failed += 1
                failed_cases.append({
                    'text': text,
                    'expected': expected,
                    'got': result,
                    'missing': list(missing),
                    'extra': list(extra)
                })
                print(f"FAIL: '{text}'")
                print(f"  Expected: {expected}")
                print(f"  Got:      {result}")
                if missing:
                    print(f"  Missing:  {list(missing)}")
                if extra:
                    print(f"  Extra:    {list(extra)}")
                    
        except Exception as e:
            failed += 1
            failed_cases.append({
                'text': text,
                'error': str(e)
            })
            print(f"ERROR processing: '{text}' - {str(e)}")
    
    # Print summary
    print(f"\nTest Results: {passed} passed, {failed} failed")
    
    if failed_cases:
        print("\nFailed Cases Summary:")
        for case in failed_cases:
            if 'error' in case:
                print(f"Text: '{case['text']}'")
                print(f"Error: {case['error']}")
            else:
                print(f"Text: '{case['text']}'")
                print(f"Expected: {case['expected']}")
                print(f"Got:      {case['got']}")
                if case['missing']:
                    print(f"Missing:  {case['missing']}")
                if case['extra']:
                    print(f"Extra:    {case['extra']}")
            print("---")
    
    return {
        'passed': passed,
        'failed': failed,
        'failed_cases': failed_cases,
        'success_rate': passed / (passed + failed) if (passed + failed) > 0 else 0
    }

In [34]:
# skills_list = [skill for synonyms in STANDARD_SKILL_MAP.values() for skill in synonyms]
extractor = SkillExtractor(STANDARD_SKILL_MAP)
text = 'MLOps são importantes para a vaga.'
extractor.extract_skills(text)

2025-05-18 17:30:52,946 - src.analysis.extracting_skills_list - SUCCESS - Successfully loaded spaCy model: pt_core_news_lg
2025-05-18 17:30:52,948 - src.analysis.extracting_skills_list - INFO - Prepared regex patterns for 126 skills.


['MLOps']

In [31]:
test_skill_extractor()

2025-05-18 17:30:30,647 - src.analysis.extracting_skills_list - SUCCESS - Successfully loaded spaCy model: pt_core_news_lg
2025-05-18 17:30:30,648 - src.analysis.extracting_skills_list - INFO - Prepared regex patterns for 126 skills.


set()
PASS: 'Precisamos de um profissional com experiência em Python e SQL.' -> ['SQL', 'Python']
set()
PASS: 'Conhecimento em Machine Learning (ML) e PowerBI é essencial.' -> ['Power BI', 'Machine Learning']
set()
PASS: 'Domínio de AWS, Azure e GCP para cloud computing.' -> ['AWS', 'GCP', 'Azure', 'Cloud']
set()
PASS: 'Experiência com PySpark e Spark para processamento de dados.' -> ['Spark']
set()
PASS: 'Habilidade em Excel avançado e análise de dados.' -> ['Excel']
set()
PASS: 'Conhecimento em Airflow para orquestração de pipelines.' -> ['Airflow']
set()
PASS: 'Não há requisitos técnicos específicos para esta vaga.' -> []
set()
PASS: 'Necessário conhecimento em Python e pandas para análise de dados.' -> ['Pandas', 'Python']
set()
PASS: 'A palavra 'excelente' não deve ser confundida com 3xc3l.' -> []
{'MLOps'}
FAIL: 'MLOps e Machine Learning são importantes para a vaga.'
  Expected: ['Machine Learning']
  Got:      ['Machine Learning', 'MLOps']
  Extra:    ['MLOps']

Test Results: 9 

{'passed': 9,
 'failed': 1,
 'failed_cases': [{'text': 'MLOps e Machine Learning são importantes para a vaga.',
   'expected': ['Machine Learning'],
   'got': ['Machine Learning', 'MLOps'],
   'missing': [],
   'extra': ['MLOps']}],
 'success_rate': 0.9}

---

# Titles

In [1]:
import sys
import os
import pandas as pd

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Importing from src
from config.analysis import ROLE_PATTERNS
from src.analysis.analysis_utils import test_classifier, title_classifier
from src.utils.logger import setup_logging

setup_logging()


In [2]:
list_positions = ['Pessoa Engenheira de Dados Sr - Outros Dados', '[Dados] Especialista em Análise de Dados - Outros Dados', 
                  'Artificial Intelligence Engineer - Outros', 'Artificial Intelligence Engineer, IgniteTech (... - Outros',
                  'Banco de Talentos - Eng de Dados - Júnior/Plen... - Outros Dados', 'Pessoa Engenheira de Dados Sênior - Outros Dados',
                  'Data Insights Analyst - Outros Dados']

for position in list_positions:
    print(f'{position}: {title_classifier(position)}')

Pessoa Engenheira de Dados Sr - Outros Dados: Engenheiro de Dados
[Dados] Especialista em Análise de Dados - Outros Dados: Analista de Dados
Artificial Intelligence Engineer - Outros: Engenheiro de IA
Artificial Intelligence Engineer, IgniteTech (... - Outros: Engenheiro de IA
Banco de Talentos - Eng de Dados - Júnior/Plen... - Outros Dados: Engenheiro de Dados
Pessoa Engenheira de Dados Sênior - Outros Dados: Engenheiro de Dados
Data Insights Analyst - Outros Dados: Analista de Dados


In [4]:
jobs_classified = pd.read_csv('../data/processed/df_jobs_classified.csv')
jobs_classified_tuple = list(zip(jobs_classified['job_title'], jobs_classified['classified_job_title']))

test_classifier(jobs_classified_tuple)

2025-06-17 19:43:45,354 - src.analysis.analysis_utils - INFO - Starting classifier test...
2025-06-17 19:43:45,354 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Data & AI Analyst' | Expected: 'Outros Dados', Got: 'Analista de Dados'
2025-06-17 19:43:45,368 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Especialista de operação de Dados - Vaga Afirmativa Feminino | Jurubatuba SP' | Expected: 'Outros Dados', Got: 'Analista de Dados'
2025-06-17 19:43:45,399 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados (Sr/Especialista)' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados'
2025-06-17 19:43:45,400 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados
Trabalho Remoto
Efetivo' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados'
2025-06-17 19:43:45,402 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados Sr
Trabalho Remoto
Efetivo' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados

FAIL Case 27: Input Title: 'Data & AI Analyst'
  Expected: 'Outros Dados'
  Got:      'Analista de Dados'
FAIL Case 371: Input Title: 'Especialista de operação de Dados - Vaga Afirmativa Feminino | Jurubatuba SP'
  Expected: 'Outros Dados'
  Got:      'Analista de Dados'
FAIL Case 1378: Input Title: 'Pessoa Engenheira de Dados (Sr/Especialista)'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 1386: Input Title: 'Pessoa Engenheira de Dados
Trabalho Remoto
Efetivo'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 1427: Input Title: 'Pessoa Engenheira de Dados Sr
Trabalho Remoto
Efetivo'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 1470: Input Title: 'Pessoa Engenheira de Dados'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 1473: Input Title: 'Pessoa Engenheira de Dados Sr'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 1475: Input Title: 'Pessoa Engenheira de Dados Sêni

2025-06-17 19:43:45,535 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Especialista de Dados' | Expected: 'Outros Dados', Got: 'Analista de Dados'
2025-06-17 19:43:45,535 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Especialista em Dados - Python' | Expected: 'Outros Dados', Got: 'Analista de Dados'
2025-06-17 19:43:45,535 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados Pleno [AWS]' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados'
2025-06-17 19:43:45,535 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados Pleno' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados'
2025-06-17 19:43:45,543 - src.analysis.analysis_utils - WARNING - Test FAIL: 'AI Software Engineer (Remote)' | Expected: 'Outros', Got: 'Engenheiro de IA'
2025-06-17 19:43:45,548 - src.analysis.analysis_utils - WARNING - Test FAIL: 'Pessoa Engenheira de Dados Sênior' | Expected: 'Outros Dados', Got: 'Engenheiro de Dados'
2025-06-17 19:43:45,

FAIL Case 5115: Input Title: 'Especialista de Dados'
  Expected: 'Outros Dados'
  Got:      'Analista de Dados'
FAIL Case 5132: Input Title: 'Especialista em Dados - Python'
  Expected: 'Outros Dados'
  Got:      'Analista de Dados'
FAIL Case 5136: Input Title: 'Pessoa Engenheira de Dados Pleno [AWS]'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 5140: Input Title: 'Pessoa Engenheira de Dados Pleno'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 5341: Input Title: 'AI Software Engineer (Remote)'
  Expected: 'Outros'
  Got:      'Engenheiro de IA'
FAIL Case 5461: Input Title: 'Pessoa Engenheira de Dados Sênior'
  Expected: 'Outros Dados'
  Got:      'Engenheiro de Dados'
FAIL Case 5472: Input Title: 'IA Engineer Mid-Level'
  Expected: 'Outros'
  Got:      'Engenheiro de IA'
FAIL Case 5717: Input Title: 'Especialista em Operação de Dados'
  Expected: 'Outros Dados'
  Got:      'Analista de Dados'
FAIL Case 5778: Input Title: 'Data Insights

In [5]:
jobs_classified[jobs_classified['classified_job_title']=='Engenheiro de Dados'][['job_title', 'classified_job_title']].sample(20)

,job_title,classified_job_title
2344,Data Engineer SR - Apache Flink,Engenheiro de Dados
7273,Engenheiro de Dados AWS Sênior,Engenheiro de Dados
2404,Staff Data Engineer - User Knowledge,Engenheiro de Dados
3155,ENGENHEIRO DE DADOS II,Engenheiro de Dados
7457,ANALISTA COMERCIAL SR (ENGENHEIRO DE DADOS),Engenheiro de Dados
6699,Engenheiro de dados senior,Engenheiro de Dados
3127,Engenheiro de Dados,Engenheiro de Dados
7225,Data Engineer Senior,Engenheiro de Dados
2333,Data Engineer,Engenheiro de Dados
6125,Engenheiro de Dados Sr,Engenheiro de Dados


---

# Location

In [4]:
import pandas as pd
import re


In [18]:
def standardize_locations(df: pd.DataFrame, location_col: str = 'location') -> pd.DataFrame:
    """
    Standardizes location data into separate city, state, and country columns.
    Handles cases like "São Paulo, Brasil" and "Distrito Federal, Brasil" correctly.
    Also handles "e Região" patterns for state capitals.
    """
    
    df['city'] = None
    df['state'] = None
    df['country'] = 'Brasil'
    
    brazilian_states = {
        'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
        'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal',
        'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
        'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
        'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
        'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
        'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
        'RS': 'Rio Grande do Sul', 'RO': 'Rondônia',
        'RR': 'Roraima', 'SC': 'Santa Catarina',
        'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
    }
    
    state_capitals = {
        'Rio Branco': 'AC', 'Maceió': 'AL', 'Macapá': 'AP', 'Manaus': 'AM',
        'Salvador': 'BA', 'Fortaleza': 'CE', 'Brasília': 'DF', 'Vitória': 'ES',
        'Goiânia': 'GO', 'São Luís': 'MA', 'Cuiabá': 'MT', 'Campo Grande': 'MS',
        'Belo Horizonte': 'MG', 'Belém': 'PA', 'João Pessoa': 'PB', 'Curitiba': 'PR',
        'Recife': 'PE', 'Teresina': 'PI', 'Rio De Janeiro': 'RJ', 'Natal': 'RN',
        'Porto Alegre': 'RS', 'Porto Velho': 'RO', 'Boa Vista': 'RR', 'Florianópolis': 'SC',
        'São Paulo': 'SP', 'Aracaju': 'SE', 'Palmas': 'TO'
    }
    
    state_mapping = {**{v: k for k, v in brazilian_states.items()}, **brazilian_states}
    
    state_names = set(brazilian_states.values())
    state_abbrevs = set(brazilian_states.keys())
    
    def extract_location(location):
        if pd.isna(location):
            return (None, None, None)
            
        location = str(location).strip()

        if location.lower() in ['brasil']:
            return (None, None, 'Brasil')
        
        if location in state_names or location in state_abbrevs:
            state_code = location if location in state_abbrevs else state_mapping.get(location)
            return (None, state_code, 'Brasil')
            
        match = re.match(r'^(?P<state>[^,]+),\s*Brasil$', location, re.IGNORECASE)
        if match and match.group('state') in state_names:
            state_name = match.group('state')
            return (None, state_mapping.get(state_name), 'Brasil')
        
        match = re.match(r'^(?P<city>.+)\s+e\s+Região$', location)
        if match:
            city = match.group('city').strip().title()
            state = state_capitals.get(city)
            return (city, state, 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>[A-Z]{2})$', location)
        if match:
            return (match.group('city').title(), match.group('state').upper(), 'Brasil')
        
        match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>.+)$', location)
        if match:
            state = match.group('state').strip()
            
            if state.lower() in ['brasil', 'brazil']:
                city_name = match.group('city').title()
                return (city_name, state_capitals.get(city_name), 'Brasil')
                
            if state in state_names or state in state_abbrevs:
                state_code = state if state in state_abbrevs else state_mapping.get(state)
                return (match.group('city').title(), state_code, 'Brasil')
            
            return (match.group('city').title(), None, 'Brasil')
        
        city_name = location.title()
        if city_name in state_capitals:
            return (city_name, state_capitals.get(city_name), 'Brasil')
        
        return (city_name, None, 'Brasil')
    
    df[['city', 'state', 'country']] = df[location_col].apply(
        lambda x: pd.Series(extract_location(x))
    )
    
    return df

In [19]:
brazilian_states = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
    'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal',
    'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
    'MT': 'Mato Grosso', 'MS': 'Mato Grosso Do Sul',
    'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
    'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
    'RJ': 'RioDe Janeiro', 'RN': 'Rio Grande Do Norte',
    'RS': 'Rio Grande Do Sul', 'RO': 'Rondônia',
    'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}

# Brazilian state capitals mapping (capital city: state abbreviation)
state_capitals = {
    'Rio Branco': 'AC', 'Maceió': 'AL', 'Macapá': 'AP', 'Manaus': 'AM',
    'Salvador': 'BA', 'Fortaleza': 'CE', 'Brasília': 'DF', 'Vitória': 'ES',
    'Goiânia': 'GO', 'São Luís': 'MA', 'Cuiabá': 'MT', 'Campo Grande': 'MS',
    'Belo Horizonte': 'MG', 'Belém': 'PA', 'João Pessoa': 'PB', 'Curitiba': 'PR',
    'Recife': 'PE', 'Teresina': 'PI', 'Rio De Janeiro': 'RJ', 'Natal': 'RN',
    'Porto Alegre': 'RS', 'Porto Velho': 'RO', 'Boa Vista': 'RR', 'Florianópolis': 'SC',
    'São Paulo': 'SP', 'Aracaju': 'SE', 'Palmas': 'TO'
}

# Create bidirectional mapping for state names and abbreviations
state_mapping = {**{v: k for k, v in brazilian_states.items()}, **brazilian_states}

    # Set of state names (to check if a location is a state)
state_names = set(brazilian_states.values())
state_abbrevs = set(brazilian_states.keys())

def extract_location(location):
    if pd.isna(location):
        return (None, None, None)
        
    location = str(location).strip()
    
    # Handle country-level or remote cases
    if location.lower() in ['brasil', 'brazil', 'remote', 'remoto']:
        return (None, None, 'Brasil')
    
    # Handle special case of "Distrito Federal, Brasil" and other direct state references
    if location in state_names or location in state_abbrevs:
        state_code = location if location in state_abbrevs else state_mapping.get(location)
        return (None, state_code, 'Brasil')
        
    # Check for state directly followed by country ("Distrito Federal, Brasil")
    match = re.match(r'^(?P<state>[^,]+),\s*Brasil$', location, re.IGNORECASE)
    if match and match.group('state') in state_names:
        state_name = match.group('state')
        return (None, state_mapping.get(state_name), 'Brasil')
    
    # Pattern for "City e Região" (e.g., "São Paulo e Região")
    # Important: What comes with "e Região" is ALWAYS the city
    match = re.match(r'^(?P<city>.+)\s+e\s+Região$', location)
    if match:
        city = match.group('city').strip().title()
        # Get state from the capital mapping
        state = state_capitals.get(city)
        return (city, state, 'Brasil')
    
    # Pattern for "City, ST" (e.g., "São Paulo, SP")
    match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>[A-Z]{2})$', location)
    if match:
        return (match.group('city').title(), match.group('state').upper(), 'Brasil')
    
    # Pattern for "City, State" (e.g., "São Paulo, São Paulo")
    match = re.match(r'^(?P<city>[^,]+),\s*(?P<state>.+)$', location)
    if match:
        state = match.group('state').strip()
        
        # Check if state is actually a country reference
        if state.lower() in ['brasil', 'brazil']:
            city_name = match.group('city').title()
            # Check if city is a state capital
            return (city_name, state_capitals.get(city_name), 'Brasil')
            
        # Check if the supposed state is actually a state
        if state in state_names or state in state_abbrevs:
            state_code = state if state in state_abbrevs else state_mapping.get(state)
            return (match.group('city').title(), state_code, 'Brasil')
        
        # Otherwise assume first part is city, second part unknown
        return (match.group('city').title(), None, 'Brasil')
    
    city_name = location.title()
    # Check if the city is a state capital
    if city_name in state_capitals:
        return (city_name, state_capitals.get(city_name), 'Brasil')
    
    return (city_name, None, 'Brasil')

In [22]:
dict_test = {'location': ['Rio Verde, Goiás, Brasil']}
df = pd.DataFrame(dict_test)
standardize_locations(df, 'location')

,location,city,state,country
0,"Rio Verde, Goiás, Brasil",Rio Verde,None,Brasil


In [7]:
extract_location('Brasil')

(None, None, 'Brasil')

In [27]:
def test_location(test_cases):
    """Test function to verify classifier behavior"""

    failed = 0
    for location, expected in test_cases:
        result = extract_location(location)
        if result != expected:
            print(f"FAIL: '{location}'")
            print(f"  Expected: {expected}")
            print(f"  Got:      {result}")
            failed += 1

    print(f"\nTest results: {len(test_cases)-failed} passed, {failed} failed")

In [34]:
test_cases = [
    ('São Paulo, SP', ('São Paulo', 'SP', 'Brasil')),
    ('Brasil', (None, None, 'Brasil')),
    ('São Paulo e Região', ('São Paulo', 'SP', 'Brasil')),
    ('Rio de Janeiro e Região', ('Rio De Janeiro', 'RJ', 'Brasil')),
    ('Distrito Federal, Brasil', (None, 'DF', 'Brasil')),
    ('Naviraí, MS', ('Naviraí', 'MS', 'Brasil'))
    ]

In [35]:
test_location(test_cases)


Test results: 6 passed, 0 failed


In [38]:
jobs = pd.read_csv('../data/raw/jobs_data.csv')
job_copy = jobs.copy()
df = standardize_locations(jobs, location_col='location')

In [39]:
df[df['location'].str.contains('remoto')]

,job_id,work_model,keyword,scrape_date,job_title,company_name,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description,city,state,country


In [41]:
df['city'].unique()

array(['Barueri', 'São Paulo', 'Curitiba', 'Maracanaú', 'Brasília',
       'Guará', 'Campinas', 'Fortaleza', 'Blumenau', 'Indaial',
       'Rio De Janeiro', 'Porto Alegre', 'Campo Grande', 'Contagem',
       'Mogi Das Cruzes', 'Uberlândia', 'Caxias Do Sul', 'Cravinhos',
       'Pereiro', 'Macaé', 'Itajaí', 'Osasco', 'Betim', 'Palotina', None,
       'Goiânia', 'Serra', 'Novo Hamburgo', 'Tijucas', 'Maringá',
       'Cachoeirinha', 'Joinville', 'Vitória', 'Sorocaba',
       'Ribeirão Preto', 'Mossoró', 'Marília', 'Jaraguá Do Sul', 'Arcos',
       'Dois Irmãos', 'São Bernardo Do Campo', 'Várzea Grande',
       'Belo Horizonte', 'Aracaju', 'Pato Branco', 'Franca', 'Rio Do Sul',
       'Ibirama', 'Manaus', 'Recife', 'Embu Das Artes', 'Eldorado Do Sul',
       'Viçosa', 'Leopoldina', 'Monte Belo', 'Vila Velha', 'Londrina',
       'Salvador', 'Naviraí', 'Morrinhos', 'Cuiabá', 'Florianópolis',
       'Jundiaí', 'Belém', 'Teresina', 'São José Do Rio Preto',
       'Santo André', 'Santa Rosa', '

In [45]:
# mask = (job_copy[['city', 'state', 'country']] != df[['city', 'state', 'country']])
# # Show both DataFrames side by side where they differ
# differences = pd.concat([job_copy[mask.any(axis=1)], df[mask.any(axis=1)]], axis=1, keys=['job_copy', 'df'])
# differences[[('job_copy', 'city'), ('job_copy', 'state'), ('job_copy', 'country'), ('df', 'city'), ('df', 'state'), ('df', 'country')]].sample(10)